<p style="text-align:center">
        <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
</p>


### Analyse search terms on the e-commerce web server


##### In this assignment you will download the search term data set for the e-commerce web server and run analytic queries on it.


In [ ]:
# Install spark

In [1]:
!pip install pyspark
!pip install findspark

In [2]:
import findspark
findspark.init()

In [3]:
import tarfile
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.regression import LinearRegressionModel
from pyspark.ml.feature import VectorAssembler

In [ ]:
# Start session

In [4]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("SparkML Ops").getOrCreate()

23/08/31 22:08:39 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
# Download The search term dataset from the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

In [ ]:
!curl https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv >> searchterms.csv

In [ ]:
# Load the csv into a spark dataframe

In [5]:
df = spark.read.csv("searchterms.csv", header=True, inferSchema=True)

In [ ]:
# Print the number of rows and columns
# Take a screenshot of the code and name it as shape.jpg)

In [6]:
num_rows = df.count()
num_columns = len(df.columns)

print("Number of rows:", num_rows)
print("Number of columns:", num_columns)

Number of rows: 10000
Number of columns: 4


In [ ]:
# Print the top 5 rows
# Take a screenshot of the code and name it as top5rows.jpg)

In [7]:
df.show(5)

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows



In [ ]:
#Find out the datatype of the column searchterm ?
# Take a screenshot of the code and name it as datatype.jpg)

In [11]:
df.printSchema

<bound method DataFrame.printSchema of DataFrame[day: int, month: int, year: int, searchterm: string]>

In [ ]:
# How many times was the term `gaming laptop` searched?
# Take a screenshot of the code and name it as gaminglaptop.jpg)

In [9]:
search_term = "gaming laptop"
search_count = df.filter(col("searchterm") == search_term).count()
print(f"The term '{search_term}' was searched {search_count} times.")

The term 'gaming laptop' was searched 499 times.


In [ ]:
# Print the top 5 most frequently used search terms?
# Take a screenshot of the code and name it as top5terms.jpg)

In [10]:
top_search_terms = df.groupBy(["searchterm"]).agg({"searchterm":"count"}).sort("count(searchterm)", ascending = False).show(5)

[Stage 8:==================================================>    (184 + 8) / 200]

+-------------+-----------------+
|   searchterm|count(searchterm)|
+-------------+-----------------+
|mobile 6 inch|             2312|
|    mobile 5g|             2301|
|mobile latest|             1327|
|       laptop|              935|
|  tablet wifi|              896|
+-------------+-----------------+
only showing top 5 rows



In [ ]:
import tarfile
from pyspark.ml.regression import LinearRegressionModel
from pyspark.ml.feature import VectorAssembler


In [ ]:
# The pretrained sales forecasting model is available at  the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz

In [ ]:
# Load the sales forecast model.
# Take a screenshot of the code and name it as loadmodel.jpg)
import tarfile
from pyspark.ml.regression import LinearRegressionModel
!curl https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz >> model.tar.gz


In [12]:
model_archive_path = "model.tar.gz"
with tarfile.open(model_archive_path, "r:gz") as tar:
    tar.extractall()

model = LinearRegressionModel.load('sales_prediction.model')


In [ ]:
# Using the sales forecast model, predict the sales for the year of 2023.
# Take a screenshot of the code and name it as forecast.jpg

In [ ]:
df.show()

In [ ]:
assembler = VectorAssembler(
    inputCols=["year"],
    outputCol="features")

data = assembler.transform(df).select('features','year')

In [ ]:
data.show()

In [13]:
def predict_sales(year_value):
    assembler = VectorAssembler(inputCols=["year"], outputCol="features")
    data = [[year_value]]
    columns = ["year"]
    data_df = spark.createDataFrame(data, columns)
    data_with_features = assembler.transform(data_df)
    predictions = model.transform(data_with_features)
    predictions.select("year", "prediction").show()



In [14]:
predict_sales(2023)

+----+------------------+
|year|        prediction|
+----+------------------+
|2023|175.16564294006457|
+----+------------------+



23/08/31 22:12:18 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/08/31 22:12:18 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
